In [1]:
import tensorflow as tf
import numpy as np
import time
import os
os.environ['CUDA_VISIBLE_DEVICES']='6'

In [2]:
from keras.datasets import mnist, cifar10
(img_train, label_train), (img_test, label_test) = cifar10.load_data()
from keras.utils import np_utils

row, col = img_train.shape[1], img_train.shape[2]

if len(img_train.shape) == 4 :
    depth = img_train.shape[3]
else:
    depth = 1
    img_train = img_train.reshape(img_train.shape[0], row, col, depth)
    img_test = img_test.reshape(img_test.shape[0], row, col, depth)

img_train = img_train.astype('float32')
img_test = img_test.astype('float32')
img_train /= 255
img_test /= 255

img_train = img_train - np.mean(img_train, axis=0)
img_test = img_test - np.mean(img_test, axis= 0)

img_train = img_train / np.var(img_train, axis=0)
img_test = img_test / np.var(img_test, axis =0)

label_train = np_utils.to_categorical(label_train, 10)
label_test = np_utils.to_categorical(label_test, 10)


Using TensorFlow backend.


In [3]:
init_size = 4
batch_size = 256
learning_rate = 0.001
epochs = 1000

In [4]:
class layer:
    def __init__(self):
        pass
        
    def block(self, input_, filters, size):
        for idx in range(size):
            L = conv(input_, filters)
                     
        L_max = tf.layers.max_pooling2d(L, pool_size=(2, 2), strides=(2, 2), padding='same')
                        
        return L_max       
            
def conv(input_, filters, kernel_size=(3,3), strides=(1,1), padding='same'):
    L = tf.layers.conv2d(input_, filters=filters, kernel_size=kernel_size, 
                         strides=strides, padding=padding)
    L_BN = tf.layers.batch_normalization(L, axis=-1)
    L_BN = tf.nn.relu(L_BN)
    
    return L_BN
    

In [5]:
X = tf.placeholder(tf.float32, [None, row, col, depth])
Y = tf.placeholder(tf.float32, [None, 10])

layers = layer()
tower1 = layers.block(X, init_size, size=2)
tower2 = layers.block(tower1, init_size*2, size=2)
tower3 = layers.block(tower2, init_size*4, size=2)
#tower4 = layers.block(tower3, init_size*4, init_size*8, size=3)
#tower5 = layers.block(tower4, init_size*8, init_size*8, size=3)

In [6]:
tower3

<tf.Tensor 'max_pooling2d_3/MaxPool:0' shape=(?, 4, 4, 16) dtype=float32>

In [7]:
flat_shape = 4*4*init_size*4

flat = tf.reshape(tower3, [-1, flat_shape])

L = tf.layers.dense(flat, units=init_size*8)
L_BN = tf.layers.batch_normalization(L, axis=-1)
L_BN = tf.nn.relu(L_BN)
logits = tf.layers.dense(L_BN, units=label_test.shape[1])

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
print ('Learning Started.')
for epoch in range(epochs):
    start = time.time()
    avg_cost = 0
    total_batch = int(img_train.shape[0] / batch_size)
    
    for i in range(total_batch):
        idxs = np.random.permutation(xrange(len(img_train)))
        idxs_i = idxs[i * batch_size : (i + 1) * batch_size]
        feed_dict = {X: img_train[idxs_i], Y: label_train[idxs_i]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print '=================================='
    print 'Epoch: {:04d}'.format(epoch + 1), 'cost = {:.6f}'.format(avg_cost)
    print 'Train Accuracy: {:.6f}'.format(sess.run(accuracy, 
                                             feed_dict={X: img_train, Y: label_train}))
    print 'Test Accuracy: {:.6f}'.format(sess.run(accuracy, 
                                             feed_dict={X: img_test, Y: label_test}))
    end = time.time()
    print 'Elapsed time: {:.6f} sec'.format(end-start)

Learning Started.
Epoch: 0001 cost = 2.002115
Train Accuracy: 0.398020
Test Accuracy: 0.393700
Elapsed time: 64.577423 sec


KeyboardInterrupt: 

In [9]:
tf.trainable_variables()

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 4) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 3, 4) dtype=float32_ref>,
 <tf.Variable 'conv2d_1/bias:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 4, 8) dtype=float32_ref>,
 <tf.Variable 'conv2d_2/bias:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_2/beta:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_2/gamma:0' shape=(8,) dtype=float32_ref>,
 <tf.Variable 'conv2d_3/kernel:0' shape=(3, 3, 4, 8) dtype=float32_ref>,
 <tf.Variable 'conv2d_3/bias:0' shape=(8,) dtype=float32_ref>,
 <tf.

In [ ]:
print ('Learning Started.')
for epoch in range(epochs):
    start = time.time()
    avg_cost = 0
    total_batch = int(img_train.shape[0] / batch_size)
    
    for i in range(total_batch):
        idxs = np.random.permutation(xrange(len(img_train)))
        idxs_i = idxs[i * batch_size : (i + 1) * batch_size]
        feed_dict = {X: img_train[idxs_i], Y: label_train[idxs_i]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print '=================================='
    print 'Epoch: {:04d}'.format(epoch + 1), 'cost = {:.6f}'.format(avg_cost)
    print 'Train Accuracy: {:.6f}'.format(sess.run(accuracy, 
                                             feed_dict={X: img_train, Y: label_train}))
    print 'Test Accuracy: {:.6f}'.format(sess.run(accuracy, 
                                             feed_dict={X: img_test, Y: label_test}))
    end = time.time()
    print 'Elapsed time: {:.6f} sec'.format(end-start)

Learning Started.
Epoch: 0001 cost = 447.175734
Train Accuracy: 0.497060
Test Accuracy: 0.485600
Elapsed time: 10.529827 sec
Epoch: 0002 cost = 339.443727
Train Accuracy: 0.562660
Test Accuracy: 0.546500
Elapsed time: 5.688746 sec
Epoch: 0003 cost = 305.182606
Train Accuracy: 0.604860
Test Accuracy: 0.584000
Elapsed time: 5.760675 sec
Epoch: 0004 cost = 282.816185
Train Accuracy: 0.629520
Test Accuracy: 0.601400
Elapsed time: 5.842239 sec
Epoch: 0005 cost = 269.537920
Train Accuracy: 0.640040
Test Accuracy: 0.603900
Elapsed time: 5.708483 sec
Epoch: 0006 cost = 256.225136
Train Accuracy: 0.660780
Test Accuracy: 0.620700
Elapsed time: 5.697974 sec
Epoch: 0007 cost = 246.113661
Train Accuracy: 0.676260
Test Accuracy: 0.635000
Elapsed time: 5.849974 sec
Epoch: 0008 cost = 238.300833
Train Accuracy: 0.690540
Test Accuracy: 0.641300
Elapsed time: 5.895105 sec
Epoch: 0009 cost = 228.454985
Train Accuracy: 0.697380
Test Accuracy: 0.650600
Elapsed time: 5.848239 sec
Epoch: 0010 cost = 221.3044

In [ ]:
print ('Accuracy:', sess.run(accuracy, feed_dict={X: img_test, Y: label_test}))

In [24]:

# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
#X_img = tf.reshape(X, [-1, 28, 28, 1])   # img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32, [None, 10])

# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
'''

# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64])
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
'''

# Final FC 7x7x64 inputs -> 10 outputs
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(img_train.shape[0] / batch_size)

    for i in range(total_batch):
        idxs = np.random.permutation(range(len(img_train)))
        idxs_i = idxs[i * batch_size: (i + 1) * batch_size]
        feed_dict = {X: img_train[idxs_i], Y: label_train[idxs_i]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy:', sess.run(accuracy, feed_dict={
      X: img_test, Y: label_test}))
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy


Learning started. It takes sometime.
('Accuracy:', 0.96750015)
('Epoch:', '0001', 'cost =', '0.317009241')
('Accuracy:', 0.97760016)
('Epoch:', '0002', 'cost =', '0.090142726')
('Accuracy:', 0.9825002)
('Epoch:', '0003', 'cost =', '0.070426149')
('Accuracy:', 0.98270005)
('Epoch:', '0004', 'cost =', '0.056876211')


KeyboardInterrupt: 